<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/NEXRAD_Level2_Download_and_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="+4" color="green"><b>Plot NexRad images</b></font>  
<font size="+1" color="gray"><i>updated June 16, 2025</i></font>

---
Questions or bugs? Please contact me at thomas.turnage@noaa.gov

---

<font size="+2">**Allows the user to select NEXRAD Level2 data to download and plot it**</font>  
<font size="+1">*Currently just plots reflectivity*</font>
<!--
<font size="+1">*The code borrows heavily from:*</font>
<ul>
<li>this <a href="https://lsterzinger.medium.com/add-lat-lon-coordinates-to-goes-16-goes-17-l2-data-and-plot-with-cartopy-27f07879157f" target="_blank">medium article</a> showing how to slice satellite data by lat/lon coordinates</li>
<li>The <a href="https://goes2go.readthedocs.io/en/latest/" target="_blank">goes2go</a> library developed by <a href="https://github.com/blaylockbk" target="_blank">Brian Blaylock</a> including <a href="https://goes2go.readthedocs.io/en/latest/reference_guide/index.html#rgb-recipes" target="_blank">RGB recipes</a></li>
<li>The <a href="https://unidata.github.io/MetPy/latest/index.html" target="_blank">MetPy</a> collection of tools
</ul>
-->
<br>
<hr>

<font size="+1">*Recent Updates*</font>  
*   6/16/25: Initial Release


<hr>


In [8]:
# @title <font size="+3" color="green">Install software and datasets</font><br><i>(This takes a while but only needs to be done once! )
import sys
import os
from pathlib import Path
import shutil
import zipfile
import io
import math
import requests
from bs4 import BeautifulSoup
from datetime import timedelta
from datetime import datetime
!pip install s3fs > /dev/null 2>&1
!pip install arm-pyart > /dev/null 2>&1
import numpy as np
import s3fs
import pyart
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib as mpl
from dataclasses import dataclass

!mkdir -p /content/images


In [22]:
# @title <font size="+3" color="green">Define functions</font>

cities = {
'Grand Rapids':{'lat':42.963,'lon':-85.668, 'lat_offset':0.04, 'lon_offset':0.06},
'Muskegon':{'lat':43.234,'lon':-86.248, 'lat_offset':0.04, 'lon_offset':0.06},
'Holland':{'lat':42.7875,'lon':-86.1089, 'lat_offset':0.04, 'lon_offset':0.06},
'South Haven':{'lat':42.4031,'lon':-86.2736, 'lat_offset':0.04, 'lon_offset':0.06},
'Lansing':{'lat':42.7325,'lon':-84.5556, 'lat_offset':0.00, 'lon_offset':0.06},
'Kalamazoo':{'lat':42.292,'lon':-85.587, 'lat_offset':0.08, 'lon_offset':0.04},
'Portage':{'lat':42.20,'lon':-85.59, 'lat_offset':-0.08, 'lon_offset': 0.04},
'Ludington':{'lat':43.955,'lon':-86.4525, 'lat_offset':0.04, 'lon_offset':0.06},
'Big Rapids':{'lat':43.698,'lon':-85.4836, 'lat_offset':0.04, 'lon_offset':0.06},
'Mount Pleasant':{'lat':43.5978,'lon':-84.7675, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06},
           }

def create_and_register_cmap(cmap_name, original_colors, position, rgb_input=True) -> None:
    """
    1) Converts list of color values from 8-bit (0-255 scale)
       to 0-1 decimal scale.

    2) The created final_color_list is combine with position list
       to create a matplotlib colormap.

    3) Registers the new colormap using the supplied cmap_name.

    Input
    ----------
             cmap_name : string
                         Name of the cmap you want to use and register

      original_ colors : list
                         8-bit tuples containing RGB values
                         example: [(127,256,192), (125,125,125)]

              position : list
                         node locations for color break points on
                         a 0 to 1 scale

            rgb_input : boolean
                        True: Color conversion from RGB to performed.
                       False: Color conversion from RGB is skipped.

    Returns
    -------
    None

    """
    if rgb_input:
        final_color_list = []
        for color in original_colors:
            converted_color_group = []
            for original_channel_value in color:
                new_channel_value = float(original_channel_value)/255
                converted_color_group.append(float(format(new_channel_value, '.3f')))
            converted_color_tuple = tuple(converted_color_group)
            final_color_list.append(converted_color_tuple)
    else:
        final_color_list = original_colors

    this_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(position, final_color_list)))
    mpl.colormaps.register(cmap=this_cmap, force=True)


###############################################################################

winter_rgbs = [(0, 0, 0), (70, 70, 70), (210, 210, 210),
(0, 120, 255), (0, 0, 60), (225, 225, 0), (220, 100, 0), (220, 100, 0)]

winter_nodes = [0.0, 0.20, 0.38, 0.42, 0.48, 0.52, 0.60, 1.0]

create_and_register_cmap('winter_z', winter_rgbs, winter_nodes)

#------------------------------------------------------------------------------

wdtd_z_rgbs =[ (0, 0, 0), (0, 0, 0), (0, 0, 0), (130, 130, 130),
 (95, 189, 207), (57, 201, 105), (57, 201, 105), (0, 40, 0),
 (9, 94, 9), (255, 207, 0), (255, 207, 0), (255, 133, 0),
 (255, 0, 0), (89, 0, 0), (255, 245, 255), (225, 11, 227),
 (164, 0, 247), (99, 0, 214), (5, 221, 224), (58, 103, 181),
 (255, 255, 255),(255, 255, 255)]

def create_nodes(refs,denom,below_zero):
  nodes = []
  for r in refs:
    ref = (r + below_zero)/denom
    nodes.append(ref)
  return nodes

refs = [-32, -25.1, -25,14.99, 15,19.99, 20,34.99, 35,39.99, 40,49.99,
        50,59.99, 60,69.99, 70,74.99, 75,79.99, 80,90]

wdtd_z_nodes = create_nodes(refs,122,32)

refs = [-30, -25.1, -25, 14.99, 15,19.99, 20,34.99, 35,39.99, 40,49.99,
        50,59.99, 60,69.99, 70,74.99, 75]

# This version of wdtd z fades to white instead of black
colors=[(255, 255, 255), (130, 130, 130), (95, 189, 207), (57, 201, 105),
        (57, 201, 105), (0, 40, 0), (9, 94, 9), (255, 207, 0),
        (255, 207, 0),
        (255, 133, 0), (255, 0, 0), (89, 0, 0), (255, 245, 255),
        (225, 11, 227), (164, 0, 247), (99, 0, 214), (5, 221, 224),
        (58, 103, 181), (255, 255, 255)]
position = [0.0, 0.407, 0.409, 0.452, 0.454, 0.587, 0.590, 0.632, 0.636, 0.722,
            0.727, 0.812, 0.818, 0.902, 0.909, 0.947, 0.954, 0.992, 1.0]
create_and_register_cmap('wdtd_z', wdtd_z_rgbs, wdtd_z_nodes)
create_and_register_cmap('wdtd_new', colors, position)


class NexradLevel2():

    def __init__(self, site, start_datetime, end_datetime):
        self.site = site
        self.start_datetime = start_datetime
        self.end_datetime = end_datetime

    def daterange(self):
        """ Yields list of dates within specified time range. """

        span = self.end_datetime - self.start_datetime

        day_span = math.ceil(float(span.total_seconds()) / (3600*24))

        if self.start_datetime.hour > self.end_datetime.hour:
            day_span += 1

        for n in range(day_span):
            yield self.start_datetime + timedelta(n)

    def filelist(self):
        """ List of files for site within specified time range. """

        fs = s3fs.S3FileSystem(anon=True)
        fs.ls('s3://noaa-nexrad-level2/')

        radarfiles = []

        for single_date in self.daterange():

            YYYY = single_date.year  # strftime("%Y")
            mm = single_date.month  # strftime("%m")
            dd = single_date.day  # strftime("%d")

            # sample bucket dir : 'noaa-nexrad-level2/2018/07/19/KDMX/'
            # broken into two substrings to stay <79 columns
            bucket_str1 = f'noaa-nexrad-level2/{YYYY:.0f}/'
            bucket_str2 = f'{mm:02.0f}/{dd:02.0f}/{self.site}/'
            bucket_dir_str = bucket_str1 + bucket_str2
            # list available files in bucket
            # sample filename :  KDMX20180719_221153_V06
            files = np.array(fs.ls(bucket_dir_str))

            for f in range(0, len(files)):

                filename = files[f].split('/')[-1]  # extracts fname after /

                if 'MDM' not in filename:
                    file_date = filename[4:19]

                    try:
                        file_datetime = datetime.strptime(file_date,
                                                          '%Y%m%d_%H%M%S')

                        if (file_datetime >= self.start_datetime
                           and file_datetime <= self.end_datetime):
                            print(files[f])
                            radarfiles.append(files[f])
                            info = fs.info(files[f])

                    except Exception:
                        print("Unexpected error:", sys.exc_info()[0])

        return radarfiles

    def download(self, filelist):
        """Download level 2 radar files from AWS.

        Files will be named according to the format on AWS but assumes nothing
        about the destination folder.  self is because different users may
        have different requirements for where data should be stored.

        Args:
            filelist: A list of files in the AWS NEXRAD inventory.
            raw_data_dir: Full pathname of download destination.

        Returns:
            The return value. True for success, False otherwise.
        """
        # clean out any possible previous downloads
        !rm -rf /content/radarfiles/

        fs = s3fs.S3FileSystem(anon=True)


        download_count = 0

        # sample source filepath
        # 'noaa-nexrad-level2/2018/07/19/KDMX/KDMX20180719_221153_V06'
        for f in range(0, len(filelist)):
            try:
                #print('getting... ' + str(filelist[f]))
                dst_filepath = os.path.join('radarfiles', filelist[f].split('/')[-1])
                #print('...to ', dst_filepath)
                info = fs.info(filelist[f])
                remote_filesize = info['size']

                try:
                    stat = os.stat(dst_filepath)
                    #print(stat)
                    local_filesize = stat.st_size

                    print('remote and local filesize: ',
                          remote_filesize, local_filesize)

                    if local_filesize < remote_filesize:
                        raise Exception('File exists but is too small.')

                    print('Already downloaded.')
                    download_count += 1
                except Exception:
                    fs.get(filelist[f], dst_filepath)  # download to dest dir
                    #print('  Download complete!')
                    download_count += 1
                    # downloaded.append(filelist[f])
            except Exception:
                pass

        # Tell caller if you got all the files it wanted.
        if download_count == len(filelist):

            return True
        else:
            return False

def extract_sweeps(orig_list, cut) -> list:
    """

    Parameters
    ----------
    orig_list : list of floats representing elevations for every sweep

    cut : integer representing desired cut to plot
          possible values are 5, 9, 13, 18, 24, 35


    Returns
    -------
    cut_list : list of integers
        indices representing sweep numbers associated with CS cuts

    """
    # multiply elevations by 10 and round to ensure that the possible
    # values listed above are represented
    sweep_list = [int(round(x*10)) for x in orig_list]
    cut_list = []
    found_cut = False
    for t in range(0, len(sweep_list)):

        if sweep_list[t] == cut and found_cut is False:

            cut_list.append(t)
            found_cut = True
        elif sweep_list[t] == cut and found_cut is True:
            found_cut = False

    return cut_list


def plot_cities(plt, cities, bbox):
  buffer = 0.05
  lat_min, lat_max, lon_min, lon_max = bbox

  for key in cities:
    c = cities[key]
    lat,lon = c['lat'], c['lon']
    if ((lat > lat_min + buffer) & (lat < lat_max - buffer) & (lon > lon_min + buffer) & (lon < lon_max - buffer)):
            if 'MC' in key:
              plt.plot([lon], [lat], color='#FFAAAA', linewidth=1, marker='o', markersize=4, zorder=10)
              plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFAAAA', zorder=11)
            else:
              plt.plot([lon], [lat], color='#FFFF00', linewidth=1, marker='o', markersize=6, zorder=10)
              plt.plot([lon], [lat], color='white', linewidth=1, marker='o', markersize=4, zorder=10)
              plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, fontsize='12', color='#FFFFFF', zorder=11)


def pyart_plot_reflectivity(filepath, filename, timeshift, bbox, COUNTY_SHAPE) -> None:
    """

    Parameters
    ----------
    filename : str
        just the filename instead of the full path


    Dependencies
    ----------
    shapefile  : str
        shape_path - full local path to shapefile to add.
                     This needs to be staged in advance.

         cmap  : str
                 plts dictionary that needs to be imported.
                 Can also just use default cmap by removing vmin,vmax, cmap
                 arguments below

    Returns
    -------
    Nothing. Just exits after creating and saving plot.

    """
    (ymin, ymax, xmin, xmax) = bbox
    filepath = os.path.join('radarfiles', filename)
    this_image_dir = 'images'
    # create datetime object from filename
    file_timestamp = datetime.strptime(filename[4:19], '%Y%m%d_%H%M%S')
    # convert datetime object from UTC to local time
    local_dt_obj = file_timestamp - timedelta(hours=timeshift)
    # create string for title based on new datetime object
    title = datetime.strftime(local_dt_obj, '%a %b %d, %Y\n%I:%M %p EDT')
    radar = pyart.io.read_nexrad_archive(filepath)
    display = pyart.graph.RadarMapDisplay(radar)

    rda_lon = radar.longitude['data'][0]
    rda_lat = radar.latitude['data'][0]


    # find elevation angle associated with every sweep
    angles = list(radar.fixed_angle['data'])
    # find sweeps associated with just ~ 0.5 degrees
    desired_sweeps = extract_sweeps(angles, 5)

    for s in desired_sweeps:

        # look up first radial of the sweep
        sweep_start = radar.sweep_start_ray_index['data'][s]
        # retrieve number of seconds elapsed since very first sweep
        sweep_start_seconds = int(round(radar.time['data'][sweep_start]))
        #print(f"sweep start seconds: {sweep_start_seconds}")

        new_time_obj = file_timestamp + timedelta(seconds=sweep_start_seconds)
        file_time_title = datetime.strftime(new_time_obj, '_%Y%m%d_%H%M_UTC.png')
        #print(f"file time title: {file_time_title}")
        new_title_time_obj = new_time_obj - timedelta(hours=timeshift)
        local_time_str = datetime.strftime(new_title_time_obj, '%a %b %d, %Y\n%I:%M %p EDT')
        full_title = display.generate_title('reflectivity', s)
        # example of how temp_title looks
        # 'KILX 0.5 Deg. 2019-06-16T03:01:10.802000Z \n
        # Equivalent reflectivity factor'
        title_parts = full_title.split(' ')
        rda_str = title_parts[0]
        elevation_str = title_parts[1]
        title_head = "Grand Rapids, Michigan Radar\n"
        image_filename = rda_str + file_time_title
        title = title_head + local_time_str
        fig = plt.figure(figsize=(7, 7))
        projection = ccrs.LambertConformal(central_latitude=rda_lat,
                                           central_longitude=rda_lon)

        display.plot_ppi_map('reflectivity', int(s), vmin=-30, vmax=80,
                             cmap='wdtd_new',
                             title=title, title_flag=True,
                             colorbar_flag=True,
                             colorbar_label=' ',
                             min_lon=xmin, max_lon=xmax, min_lat=ymin,
                             max_lat=ymax,
                             projection=projection,
                             #shapefile=shape_path,
                             lat_lines=[0], lon_lines=[0],  # omit grid lines
                             fig=fig, lat_0=rda_lat, lon_0=rda_lon)

        ax = display.ax
        ax.add_feature(COUNTY_SHAPE, facecolor='none', edgecolor='gray', linewidth=0.7)
        buffer = 0.05
        lat_min, lat_max, lon_min, lon_max = bbox
        if plot_cities:
          for key in cities:
            c = cities[key]
            lat,lon = c['lat'], c['lon']
            if ((lat > lat_min + buffer) & (lat < lat_max - buffer) & (lon > lon_min + buffer) & (lon < lon_max - buffer)):
              if 'MC' in key:
                plt.plot([lon], [lat], color='#FFAAAA', linewidth=1, marker='o', markersize=4, zorder=10)
                plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFAAAA', zorder=11)
              else:
                plt.plot([lon], [lat], transform=ccrs.PlateCarree(), color='#333333', linewidth=1, marker='o', markersize=6, zorder=10)
                plt.plot([lon], [lat], transform=ccrs.PlateCarree(),color='black', linewidth=1, marker='o', markersize=4, zorder=10)
                plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, transform=ccrs.PlateCarree(), fontsize=11, color='#333333', zorder=11)

        image_dst_path = os.path.join(this_image_dir, image_filename)

        plt.savefig(image_dst_path, format='png', bbox_inches="tight", dpi=150)
        print('  Image saved at  ' + image_dst_path)
        plt.close()

shapefile_dict = {'county': {'url': 'https://www.weather.gov/gis/Counties'},
                  'us_state': {'url': 'https://www.weather.gov/gis/USStates'}}

def get_shapefiles(shape_type):
  url = shapefile_dict[shape_type]['url']
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  zip_links = soup.find_all("a", href=lambda href: href.endswith(".zip"))
  if zip_links:
    zip_url = zip_links[-1]["href"]
    this_url = os.path.join('https://www.weather.gov',str(zip_url)[1:])
    zip_file_name = this_url.split("/")[-1]
    zip_file_root = zip_file_name[:-4]
    shapefile_name = f'{zip_file_root}.shp'
  else:
    print("No zip files found on the website.")
  if not os.path.exists(f'/content/{zip_file_name}'):
    response = requests.get(this_url, stream=True)
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall('.')
    reader = shpreader.Reader(shapefile_name)
    features = list(reader.geometries())
    try:
      test = cfeature.ShapelyFeature(features, ccrs.PlateCarree())
      return test
    except:
      print("error")
      return None



In [20]:
# @title <font size="+3" color="green">Select the radar and times to acquire data for</font>
radar = "KGRR" # @param {type:"string"}

# @markdown <font size="+2" color="blue"><b><i>Select time zone for labelling your figures</i></b></font>
time_zone = "EDT" # @param ["UTC", "EST", "EDT", "CST", "CDT", "MST", "MDT", "PST", "PDT"]
time_zone_dict = {"UTC": 0, "EST":5, "EDT":4, "CST":6, "CDT":5, "MST":7, "MDT":6, "PST":8, "PDT":7 }

# @markdown <br><font color="blue"><h2>Define bounding box of the plot you want</h2></font>
lon_min = -87.5 # @param {type:"raw"}
lon_max = -83.5 # @param {type:"raw"}
lat_min = 41.5 # @param {type:"raw"}
lat_max = 44.5 # @param {type:"raw"}

def make_float(val):
  try:
    return float(val)
  except:
    print("...... WARNING !! ......................\n")
    print(f"Can't convert {val} to float!\n")
    print("...... Check values and try again ......\n")
    return

#for coor in (lat_min, lat_max, lon_min, lon_max):

ymin = make_float(lat_min)
ymax = make_float(lat_max)
xmin = make_float(lon_min)
xmax = make_float(lon_max)
bbox = (ymin, ymax, xmin, xmax)

# @markdown <br><font color="blue"><h2>Plot cities on the map?</h2></font>
plot_cities = True # @param {type:"boolean"}

# @markdown <br><font color="blue"><h2>Enter the start time (in UTC) for the radar data download</h2></font>
start_ymd = "2025-06-09" # @param {type:"date"}
start_hour = "20" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]
start_minute = "30" # @param ["00", "05", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55"]
start_time_str = f"{start_ymd} {start_hour} {start_minute}"
start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H %M")

# @markdown <br><font color="blue"><h2>Enter the duration in minutes</h2></font>
minutes_duration = "10" # @param [0,10,30,60,90,120,150,180,210,240]



end_time = start_time + timedelta(minutes=int(minutes_duration))
end_time_str = datetime.strftime(end_time, "%Y-%m-%d %H %M")

print(f"start time: {start_time} UTC")
print(f"  end time: {end_time} UTC")

example = NexradLevel2(radar, start_time, end_time)
example.download(example.filelist())
shutil.make_archive(f'{radar}_files', 'zip', 'radarfiles')


@dataclass
class PlotConfig:
  radar: str
  start_time_str: str
  end_time_str: str
  time_zone: str
  bbox: tuple
  plot_cities: bool


start time: 2025-06-09 20:30:00 UTC
  end time: 2025-06-09 20:40:00 UTC
noaa-nexrad-level2/2025/06/09/KGRR/KGRR20250609_203334_V06
noaa-nexrad-level2/2025/06/09/KGRR/KGRR20250609_203924_V06


In [24]:
# @title <font size="+3" color="green">Generate the images!</font>
try:
  COUNTY_SHAPE
except NameError:
  COUNTY_SHAPE = get_shapefiles('county')

# try:
#   STATE_SHAPE
# except NameError:
#   STATE_SHAPE = get_shapefiles('us_state')

!rm -rf images/*
for file in os.listdir('radarfiles'):
    if 'V06' in file:
        full_filepath = os.path.join('radarfiles', file)
        #print('  Beginning plot of ' + full_filepath)
        pyart_plot_reflectivity(full_filepath, file, time_zone_dict[time_zone], bbox, COUNTY_SHAPE)


  Image saved at  images/KGRR_20250609_2033_UTC.png
  Image saved at  images/KGRR_20250609_2035_UTC.png
  Image saved at  images/KGRR_20250609_2037_UTC.png
  Image saved at  images/KGRR_20250609_2039_UTC.png
  Image saved at  images/KGRR_20250609_2041_UTC.png
  Image saved at  images/KGRR_20250609_2043_UTC.png


In [ ]:
# @title <font size="+3" color="green">IGNORE: Testing Metpy plotting routines</font>

!pip install metpy > /dev/null 2>&1
import cartopy.crs as ccrs
import matplotlib.gridspec as gridspec
from metpy.calc import azimuth_range_to_lat_lon
from metpy.io import Level2File
from metpy.plots import add_metpy_logo, add_timestamp, USCOUNTIES
from metpy.units import units

In [ ]:
# @title <font size="+3" color="green">IGNORE: Testing Metpy plotting routines</font>
def get_data_array(radar_file, sweep):
  ref_hdr = f.sweeps[sweep][0][4][b'REF'][0]
  ref_range = (np.arange(ref_hdr.num_gates + 1) - 0.5) * ref_hdr.gate_width + ref_hdr.first_gate
  ref_range = units.Quantity(ref_range, 'kilometers')
  ref = np.array([ray[4][b'REF'][1] for ray in f.sweeps[sweep]])

  # Extract central longitude and latitude from file
  cent_lon = f.sweeps[0][0][1].lon
  cent_lat = f.sweeps[0][0][1].lat
  return ref, ref_range, cent_lon, cent_lat

def convert_to_lat_lon(az, ref, cent_lon, cent_lat):
  # Convert az,range to x,y
  xlocs, ylocs = azimuth_range_to_lat_lon(az, ref, cent_lon, cent_lat)
  return xlocs, ylocs


def get_azimuth_angles(radar_file):
  # First item in ray is header, which has azimuth angle
  az = np.array([ray[0].az_angle for ray in radar_file.sweeps[0]])
  diff = np.diff(az)
  crossed = diff < -180
  diff[crossed] += 360.
  avg_spacing = diff.mean()

  # Convert mid-point to edge
  az = (az[:-1] + az[1:]) / 2
  az[crossed] += 180.

  # Concatenate with overall start and end of data we calculate using the average spacing
  az = np.concatenate(([az[0] - avg_spacing], az, [az[-1] + avg_spacing]))
  az = units.Quantity(az, 'degrees')
  return az

import matplotlib.gridspec as gridspec
spec = gridspec.GridSpec(1, 1)
for file in os.listdir('radarfiles'):
    if 'V06' in file:
        full_filepath = os.path.join('radarfiles', file)
        print('  extract sweeps for ' + full_filepath)
        sweeps,filenames = pyart_detect_sweeps(full_filepath, file)
        f = Level2File(full_filepath)
        az = get_azimuth_angles(f)
        for s,fname in zip(sweeps, filenames):
          ref, ref_range, cent_lon, cent_lat = get_data_array(f, int(s))
          # Turn into an array, then mask
          data = np.ma.array(ref)
          data[np.isnan(data)] = np.ma.masked

          # Convert az,range to x,y
          xlocs, ylocs = convert_to_lat_lon(az, ref, cent_lon, cent_lat)

          # Plot the data
          fig, ax = plt.figure(figsize=(15, 8))
          crs = ccrs.LambertConformal(central_longitude=cent_lon, central_latitude=cent_lat)
          ax.add_feature(USCOUNTIES, linewidth=0.5)
          ax.pcolormesh(xlocs, ylocs, data, cmap='wdtd_new', vmin=-30, vmax=80,transform=ccrs.PlateCarree())
          ax.set_extent([cent_lon - 2, cent_lon + 2, cent_lat - 2, cent_lat + 2])
          ax.set_aspect('equal', 'datalim')
          #add_timestamp(ax, f.dt, y=0.02, high_contrast=True)

          plt.savefig(fname, format='png', bbox_inches="tight", dpi=150)
          print('  Image saved at  ' + fname)
          plt.close()

spec = gridspec.GridSpec(1, 1)
fig = plt.figure(figsize=(15, 8))

for var_data, var_range, ax_rect in zip((ref), (ref_range), spec,
                                        strict=False):
    # Turn into an array, then mask
    data = np.ma.array(var_data)
    data[np.isnan(data)] = np.ma.masked

    # Convert az,range to x,y
    xlocs, ylocs = azimuth_range_to_lat_lon(az, var_range, cent_lon, cent_lat)

    # Plot the data
    crs = ccrs.LambertConformal(central_longitude=cent_lon, central_latitude=cent_lat)
    ax = fig.add_subplot(ax_rect, projection=crs)
    ax.add_feature(USCOUNTIES, linewidth=0.5)
    ax.pcolormesh(xlocs, ylocs, data, cmap='wdtd_new', vmin=-30, vmax=80,transform=ccrs.PlateCarree())
    ax.set_extent([cent_lon - 2, cent_lon + 2, cent_lat - 2, cent_lat + 2])
    ax.set_aspect('equal', 'datalim')
    #add_timestamp(ax, f.dt, y=0.02, high_contrast=True)

plt.show()